In [4]:
import torch.nn as nn
import torch
import torch.nn.utils.rnn as rnn
import statistics
import nltk
import random
import collections
import time


In [12]:
 # Set cuda if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [13]:
## Dictionary class 선언

class Dictionary(object):
    def __init__(self, dataset, size):
        ## init vocab ##
        self.word2idx = {'<pad>':0, '<sos>': 1, '<eos>': 2, '<unk>': 3}
        self.idx2word = ['<pad>', '<sos>', '<eos>', '<unk>']

        self.build_dict(dataset, size)

    def __call__(self, word):
        return self.word2idx.get(word, self.word2idx['<unk>']) # if word does not exist in vocab then return unk idx

    def add_word(self, word):
        if word not in self.word2idx:
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def build_dict(self, dataset, dict_size):
        """Tokenize a text file."""
        total_words = (word for sent in dataset for word in sent) # store all words into tuple
        word_freq = collections.Counter(total_words) # count the number of each word: ex) ('The': 10000, 'a': 5555, ...)
        vocab = sorted(word_freq.keys(), key=lambda word: (-word_freq[word], word)) # sort by frequency
        vocab = vocab[:dict_size]
        for word in vocab:
            self.add_word(word)

    def __len__(self):
        return len(self.idx2word)


In [14]:
## Brown dataset Preprocessing (NLTK)
def brown_dataset(min=5, max=30):
    nltk.download('brown')

    # get sentences with the length between min and max
    # convert all words into lower-case
    all_seq = [[token.lower() for token in seq] for seq in nltk.corpus.brown.sents() 
               if min <= len(seq) <= max]

    random.shuffle(all_seq) # shuffle
    return all_seq

In [15]:
## Download Brown dataset
dataset = brown_dataset()

## print some part
print(dataset[0])
print(dataset[1])
print(dataset[2])

[nltk_data] Downloading package brown to /home1/wklee/nltk_data...
[nltk_data]   Package brown is already up-to-date!


['the', 'sniper', ',', 'whether', 'psychopathic', 'marksman', 'or', 'murderer', 'by', 'intent', ',', 'would', 'hardly', 'have', 'walked', 'to', 'his', 'vantage', 'point', 'with', 'rifle', 'over', 'shoulder', ',', 'whistling', 'a', 'marching', 'tune', '.']
['but', 'i', 'want', 'this', 'to', 'sink', 'in', 'awhile', '.']
['none', 'has', 'yet', 'arisen', 'under', 'condition', '(', '2', ')', ',', 'relating', 'to', 'referral', 'to', 'jobs', '``', 'the', 'filling', 'of', 'which', 'is', 'an', 'issue', 'in', 'a', 'labor', 'dispute', "''", '.']


In [16]:
## Data handler class 선언

class Corpus(object):
    def __init__(self, dataset, device, dict_size=20000, train_ratio=0.95):
        train_size = int(len(dataset) * train_ratio)
        valid_size = len(dataset) - train_size
        self.device = device
        self.dictionary = Dictionary(dataset, dict_size)
        self.train = dataset[:train_size] # 0 ~ train_size
        self.valid = dataset[train_size:train_size + valid_size] # train_size: train_size+valid_size

    def indexing(self, dat):
        src_idxes = [] # 모델 입력
        tgt_idxes = [] # 모델 정답
        for sent in dat:
            src_idx = [self.dictionary('<sos>')] + [self.dictionary(word) for word in sent]
            tgt_idx = [self.dictionary(word) for word in sent] + [self.dictionary('<eos>')]
            src_idxes.append(torch.tensor(src_idx).type(torch.int64))
            tgt_idxes.append(torch.tensor(tgt_idx).type(torch.int64))

        src_idxes = rnn.pad_sequence(src_idxes, batch_first=True).to(self.device) # shape = [B, L]
        tgt_idxes = rnn.pad_sequence(tgt_idxes, batch_first=True).to(self.device).view(-1) # flatten shape = [B * L]

        return src_idxes, tgt_idxes

    def batch_iter(self, batch_size, isTrain=True):
        dat = self.train if isTrain else self.valid
        if isTrain:
            random.shuffle(dat)

        for i in range(len(dat) // batch_size):
            batch = dat[i * batch_size: (i+1) * batch_size]
            src, tgt = self.indexing(batch)
            yield {'src': src, 'tgt': tgt}

In [30]:
corpus = Corpus(dataset, device)

In [32]:
# Dictionary 확인

for i, (key, val) in enumerate(corpus.dictionary.word2idx.items()):
    print('word:  {:10s} | index: {:5d} '.format(key, val))
    if i == 20:
        break


word:  <pad>      | index:     0 
word:  <sos>      | index:     1 
word:  <eos>      | index:     2 
word:  <unk>      | index:     3 
word:  the        | index:     4 
word:  .          | index:     5 
word:  ,          | index:     6 
word:  of         | index:     7 
word:  and        | index:     8 
word:  to         | index:     9 
word:  a          | index:    10 
word:  in         | index:    11 
word:  was        | index:    12 
word:  he         | index:    13 
word:  is         | index:    14 
word:  ''         | index:    15 
word:  ``         | index:    16 
word:  it         | index:    17 
word:  that       | index:    18 
word:  for        | index:    19 
word:  ;          | index:    20 


In [33]:
## indexing 함수 결과 확인

# case : 단일 문장 입력 시. 
sent = [dataset[1]]
idx_src, idx_tgt = corpus.indexing(sent)

print(sent)
print(idx_src) # <SOS> index로 시작
print(idx_tgt) # <EOS> index로 종료

print('-' * 90)
## case : 복수 문장 입력 시 (batching)
batch = [dataset[0], dataset[1]]
idx_src, idx_tgt = corpus.indexing(batch)

print(batch)
print(idx_src) # 가장 길이가 긴 문장 (dataset[0]) 보다 짧은 문장 (dataset[1]) 의 경우 남는 길이만큼 padding=0 삽입 확인.
print(idx_tgt)

[['but', 'i', 'want', 'this', 'to', 'sink', 'in', 'awhile', '.']]
tensor([[    1,    34,    27,   225,    28,     9,  3840,    11, 12533,     5]],
       device='cuda:0')
tensor([   34,    27,   225,    28,     9,  3840,    11, 12533,     5,     2],
       device='cuda:0')
------------------------------------------------------------------------------------------
[['the', 'sniper', ',', 'whether', 'psychopathic', 'marksman', 'or', 'murderer', 'by', 'intent', ',', 'would', 'hardly', 'have', 'walked', 'to', 'his', 'vantage', 'point', 'with', 'rifle', 'over', 'shoulder', ',', 'whistling', 'a', 'marching', 'tune', '.'], ['but', 'i', 'want', 'this', 'to', 'sink', 'in', 'awhile', '.']]
tensor([[    1,     4,     3,     6,   479, 14408,  8495,    39,  4005,    33,
          6803,     6,    49,   821,    37,   504,     9,    21, 10219,   256,
            22,  1286,    92,  1491,     6, 12220,    10,  5770, 12114,     5],
        [    1,    34,    27,   225,    28,     9,  3840,    11, 12533,   

In [34]:
## RNN Language model 선언

# Define network
class RNNModel(nn.Module):
    def __init__(self, ntoken, hidden_size, nlayers, dropout=0.1):
        super(RNNModel, self).__init__()
        self.drop = nn.Dropout(dropout)
        self.embeddings = nn.Embedding(ntoken, hidden_size, padding_idx=0)
        self.rnn = nn.LSTM(hidden_size, hidden_size, nlayers, dropout=dropout, batch_first=True) # [B, L, Hidden_DIM]
        self.output_layer = nn.Linear(hidden_size, ntoken)
        self.sm = nn.LogSoftmax(dim=1)

        self.ntoken = ntoken
        self.hidden_size = hidden_size
        self.nlayers = nlayers

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.embeddings.weight.data.uniform_(-initrange, initrange)
        self.output_layer.weight.data.uniform_(-initrange, initrange)
        self.output_layer.bias.data.zero_()

    def forward(self, input, hidden):
        emb = self.embeddings(input) # emb = (batch, length, dim)
        output, hidden = self.rnn(emb, hidden) # output = (batch. length. dim)
        output = self.drop(output)
        output = self.output_layer(output) # output = (batch, length, vocab_size)
        output = output.view(-1, self.ntoken) # output = (batch * length, vocab_size)

        return self.sm(output), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters()) # to set init tensor with the same torch.dtype and torch.device
        return (weight.new_zeros(self.nlayers, bsz, self.hidden_size),
                weight.new_zeros(self.nlayers, bsz, self.hidden_size))


In [35]:
# Hyperparameters
batch_size = 40
hidden_size = 256
dropout = 0.2
max_epoch = 30

# build model
ntokens = len(corpus.dictionary)
model = RNNModel(ntokens, hidden_size, 2, dropout).to(device)
isTrain=True # Flag variable

# set loss func and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.NLLLoss(ignore_index=0, reduction='mean')

In [ ]:
##### Training / Evaluation Parts #######

In [36]:
# accuracy
def cal_acc(scores, target):
    pred = scores.max(-1)[1]
    non_pad = target.ne(0)
    num_correct = pred.eq(target).masked_select(non_pad).sum().item() 
    num_non_pad = non_pad.sum().item()
    return 100 * (num_correct / num_non_pad)

In [37]:
# train func.
def train():
    model.train() # Turn on training mode which enables dropout.
    mean_loss = []
    mean_acc = []
    start_time = time.time()

    for batch in corpus.batch_iter(batch_size):
        hidden = model.init_hidden(batch_size) # zero vectors for init hidden
        target = batch['tgt'] # flattened target 
        optimizer.zero_grad()
        output, hidden = model(batch['src'], hidden) # output = flatten output = [Batch_size * Length, vocab_size]

        # output shape = (batch * length, vocab_size)
        # target shape = (batch * length)   --> (batch * length, vocab_size) 로 one-hot distribtuion으로 내부적으로 변환되어 비교 수행
        loss = criterion(output, target) # compare between vocab_prob and answer_prob(one-hot converted)
        loss.backward()
        optimizer.step()

        mean_loss.append(loss.item())
        mean_acc.append(cal_acc(output, target))

    total_time = time.time() - start_time
    mean_acc = statistics.mean(mean_acc)
    mean_loss = statistics.mean(mean_loss)

    return mean_loss, total_time, mean_acc

In [38]:
# evaluation func.
def evaluate():
    model.eval() # Turn off dropout
    mean_loss = []
    mean_acc = []

    for batch in corpus.batch_iter(batch_size, isTrain=False):
        with torch.no_grad():
            hidden = model.init_hidden(batch_size)
            target = batch['tgt']
            output, hidden = model(batch['src'], hidden)
            loss = criterion(output, target)
            mean_loss.append(loss.item())
            mean_acc.append(cal_acc(output, target))

    mean_acc = statistics.mean(mean_acc)
    mean_loss = statistics.mean(mean_loss)

    return mean_loss, mean_acc

In [39]:
if isTrain: # set False if you don't need to train model
    start_time = time.time()

    for epoch in range(1, max_epoch+1):
        loss, epoch_time, accuracy = train()
        print('epoch {:4d} | times {:3.3f} |  loss: {:3.3f} | accuracy: {:3.2f}'.format(epoch+1, epoch_time, loss, accuracy))

        if epoch % 10 == 0:
            loss, accuracy = evaluate()
            print('=' * 60)
            print('Evaluation | loss: {:3.3f} | accuracy: {:3.2f}'.format(epoch+1, epoch_time, loss, accuracy))
            print('=' * 60)

    with open('model.pt', 'wb') as f:
        print('save model at: ./model.pt')
        torch.save(model, f)

epoch    2 | times 13.053 |  loss: 6.091 | accuracy: 17.05
epoch    3 | times 13.071 |  loss: 5.476 | accuracy: 21.33
epoch    4 | times 13.154 |  loss: 5.235 | accuracy: 22.52
epoch    5 | times 13.191 |  loss: 5.048 | accuracy: 23.37
epoch    6 | times 13.225 |  loss: 4.890 | accuracy: 23.96
epoch    7 | times 13.235 |  loss: 4.753 | accuracy: 24.45
epoch    8 | times 13.279 |  loss: 4.633 | accuracy: 24.82
epoch    9 | times 13.267 |  loss: 4.523 | accuracy: 25.23
epoch   10 | times 13.287 |  loss: 4.427 | accuracy: 25.54
epoch   11 | times 13.289 |  loss: 4.337 | accuracy: 25.85
Evaluation | loss: 11.000 | accuracy: 13.29
epoch   12 | times 13.295 |  loss: 4.258 | accuracy: 26.19
epoch   13 | times 13.299 |  loss: 4.187 | accuracy: 26.54
epoch   14 | times 13.303 |  loss: 4.122 | accuracy: 26.95
epoch   15 | times 13.303 |  loss: 4.065 | accuracy: 27.24
epoch   16 | times 13.304 |  loss: 4.013 | accuracy: 27.52
epoch   17 | times 13.303 |  loss: 3.966 | accuracy: 27.86
epoch   18 |

/home1/wklee/envs/torch_1.0/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type RNNModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [40]:
def pred_sent_prob(sent):
    import numpy as np
    model.eval()
    with torch.no_grad():
        # 1. construct input for the model  : index 활용
        # do someting about #1....
        
        # 2. set init hidden : 첫번째 step에 대한 zero vector
        # do someting about #2....
        
        # 3. get model output : vocab 크기의 log softmax 결과값 출력
        # do someting about #3....
        
        # 4. get word log_prob : 찾고자 하는 단어의 log 확률값 추출
        # do someting about #4....
        
        # 5. comput sentence log_prob by summing each of word log_prob : 4의 각 결과를 합
        # do someting about #5....
        
        return sent_prob

In [41]:
# load saved model
with open('./model.pt', 'rb') as f:
    print('load model from: ./model.pt')
    model = torch.load(f).to(device)

    print('log prob of [the dog bark .]: {:3.3f}'.format(pred_sent_prob([['the', 'dog', 'bark', '.']])))
    print('log prob of [the cat bark .]: {:3.3f}'.format(pred_sent_prob([['the', 'cat', 'bark', '.']])))

    print('log prob of [boy am a i .]: {:3.3f}'.format(pred_sent_prob([['boy', 'am', 'a', 'i', '.']])))
    print('log prob of [i am a boy .]: {:3.3f}'.format(pred_sent_prob([['i', 'am', 'a', 'boy', '.']])))


load model from: ./model.pt
log prob of [the cat bark .]: -34.680
log prob of [the cat bark .]: -52.044
log prob of [boy am a i .]: -50.034
log prob of [i am a boy .]: -20.275
